In [18]:
import json
from typing import List

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, ConfigDict, Field

from sns_ai_automation_agency.agent.access import survey_access_information
from sns_ai_automation_agency.agent.restaurant import survey_restaurant_information

In [2]:
thread_id = "test"
station_name = "横浜"
num_highlight_stations = 3
num_iterations = 2

access_data = survey_access_information(
    station_name=station_name,
    num_highlight_stations=num_highlight_stations,
    thread_id=thread_id,
)

restaurant_data = survey_restaurant_information(
    station_name=station_name,
    num_iterations=num_iterations,
    thread_id=thread_id,
)

In [7]:
access_json = json.dumps(access_data, indent=2, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
restaurant_json = json.dumps(restaurant_data, indent=2, ensure_ascii=False).replace("{", "{{").replace("}", "}}")

In [58]:
system_prompt = """
あなたはSNSショート動画の構成作家です。  
これから渡す JSON データを解析し、  
**データ内容に応じて最適なシーン構成（シーン数は可変）** を自動生成してください。

各シーンごとに  
「タイトル・内容・テロップ・画像検索クエリ」を出力します。

---

# 🎯【生成の目的】
- 駅の「住みやすさ」「街としての魅力」を直感的に伝える
- **店舗単位ではなく、エリア・ジャンル・雰囲気を中心に紹介する**
- シーンの数はデータ量に応じて 6〜14 で柔軟に生成
- SNS向けにテンポよく、過度な専門性や冗長表現は禁止

---

# 🧠【構成のポイント（重要）】

## ■ 店舗名について（必ず守る）
- **店舗名は原則禁止**
- 店舗情報は **エリアの特徴として抽象化** して表現する  
  例：  
  - 「山が見えるテラスカフェが多いエリア」  
  - 「ラーメン屋が並ぶ路地」  
  - 「レトロな町中華が残る飲み歩きエリア」
- 施設名（大型商業施設・公園・商店街など）は使用してOK  

→ **個別店舗NG / エリア名・施設名OK / ジャンルOK**

---

# 🧩【出力仕様】


構造体の形式で出力してください: 

**Scene X**
- タイトル：
- 内容：（1〜2文、事実ベース）
- テロップ：
- 画像検索クエリ：

---

# ⚙️【生成ルール】

## ■ Scene1〜3（構造固定）
1. 導入（抽象キャッチ.駅名は入れないこと）  
2. 駅名（テロップに必ず駅名を入れること）
3. 駅全体の印象（街の雰囲気 × アクセス × エリア型の魅力）

## ■ Scene4 以降はデータ依存で動的生成
- アクセス：1〜3  
- 飲食：1〜3（店舗名NG。ジャンル/エリア/雰囲気で説明）  
- 買い物：0〜2  
- 公園：0〜2  
- 商店街/景観：任意  
- ファミリー：任意  
- まとめ：1  
- 締め：1  

---

# 🎨【テロップのルール強化】

■ 導入・景観・締め → 感情に寄せる（8〜14文字）  
例：  
- 「昼も夜も絵になる街」  
- 「海と夜景の世界へ」

■ アクセス・飲食・買い物 → 情報性を入れる（12〜18文字）  
例：  
- 「新宿へ乗換なし約15分」  
- 「駅近にカフェとスイーツ充実」  
- 「公園が徒歩圏」

■ まとめ系 → 要素を凝縮（12〜18文字）  
例：  
- 「アクセス良好で遊びも充実」

■ 共通ルール
- ナレーションで読みやすいようにすること
NG例：「カフェx景色x夜景」のように掛け算表現はナレーションで読みにくいので禁止

---

# ❗注意（最重要）
- JSON に存在しない店舗名・固有名詞は書かない  
- 個別店舗を特定できる説明は避ける  
- 必ず「エリアとしての魅力」「ジャンル」「雰囲気」でまとめる

---

最適なシーン構成とテロップを生成してください。
"""

In [59]:
user_prompt = f"""
以下の JSON データをもとに、  
上記のルールに従って SNS ショート動画のシーン構成を生成してください。

出力は「Scene 1 〜 最終シーン」まで、可変長です。

---

【入力データ】

## 主要駅へのアクセス
{access_json}

## 駅周辺の飲食店情報
{restaurant_json}
"""

In [60]:
class Scene(BaseModel):
    """シーンの構造化レスポンス"""

    model_config = ConfigDict(from_attributes=True)

    title: str = Field(..., description="シーンのタイトル")
    content: str = Field(..., description="シーンの内容")
    telop: str = Field(..., description="シーンのテロップ")
    image_search_query: str = Field(..., description="画像検索クエリ")


class MovieResponse(BaseModel):
    """動画の構造化レスポンス"""

    model_config = ConfigDict(from_attributes=True)
    scenes: List[Scene] = Field(..., description="発見したシーンのリスト")

In [61]:
scene_llm = ChatOpenAI(model="gpt-5.1", temperature=0)
scene_llm = scene_llm.with_structured_output(MovieResponse)
scene_prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

chain = scene_prompt | scene_llm
scene_output = chain.invoke({})

In [62]:
for scene in scene_output.scenes:
    print(f"タイトル: {scene.title}")
    print(f"内容: {scene.content}")
    print(f"テロップ: {scene.telop}")
    print(f"画像検索クエリ: {scene.image_search_query}")
    print("-----")

タイトル: 海と夜景のグルメターミナル
内容: 海辺の夜景も、下町の飲み屋街も、全部ぎゅっと詰まった巨大ターミナルエリアです。観光とローカル感が一度に味わえる街として人気があります。
テロップ: 海も路地も楽しめる街
画像検索クエリ: 横浜 夜景 みなとみらい 観覧車 高層ビル 夜の街並み 俯瞰
-----
タイトル: 今回歩くのは『横浜駅』周辺
内容: 舞台はJR各線が集まる横浜駅エリア。ここを起点に、みなとみらいや中華街、野毛などへサクッと足を伸ばせます。
テロップ: 今回の舞台は『横浜駅』周辺
画像検索クエリ: 横浜駅 東口 西口 駅前 ロータリー 高層ビル バスターミナル
-----
タイトル: 横浜駅の印象とアクセスの強さ
内容: 東京駅へはJR東海道本線で約25分、新宿駅へもJR湘南新宿ラインで約31分と、主要ターミナルへ乗換なしでアクセスできます。近隣の桜木町駅も約3分で、海側エリアへの移動もスムーズです。
テロップ: 東京25分・新宿31分の直通アクセス
画像検索クエリ: 横浜駅 ホーム JR東海道線 湘南新宿ライン 電車 発着 乗り換え イメージ
-----
タイトル: 東京・新宿へ直通で通勤も遊びも
内容: 東京駅へはJR東海道本線で約25分、新宿駅へはJR湘南新宿ラインで約31分と、どちらも乗換なし。品川も約12分で、都内通勤と週末のお出かけ両方に使いやすい立地です。
テロップ: 東京25分・新宿31分 乗換なし
画像検索クエリ: JR東海道線 車内 通勤客 湘南新宿ライン 路線図 首都圏 アクセス 図解
-----
タイトル: 桜木町経由で海側エリアへすぐ
内容: 横浜駅からJR京浜東北線で約3分の桜木町駅に出れば、みなとみらいや野毛エリアへ徒歩圏。海沿いの景色と飲み屋街をセットで楽しめます。
テロップ: 桜木町へ約3分で海側へ直行
画像検索クエリ: 桜木町駅 周辺地図 横浜 みなとみらい方面 アクセス 桜木町駅前 景色
-----
タイトル: 昼は海辺カフェ『みなとみらい』
内容: みなとみらいエリアは、海や観覧車を望むカフェやスイーツ店、ホテルラウンジが集まるモダンなベイエリア。横浜駅から地下鉄やみなとみらい線で直通、徒歩でも15分前後でアクセスできます。
テロップ: 海と観覧車を望むカフェエリア
画像検索クエリ: 横浜 みなとみらい カ